In [1]:
import json
import pandas as pd
from pandas import RangeIndex

In [2]:
# add dataframe into csv_to_coco folder
df = pd.read_csv('labels_my-project-name_2022-10-27-12-20-00.csv')

# add classes.txt to csv_to_coco folder
with open('classes.txt', 'r') as f:
    classes = f.readlines()
    labels = list(enumerate([i.split('\n')[0] for i in classes]))

In [3]:
# converts label names to their respective index from classes.txt
df['label_name'] = [x[0] for i in df['label_name'] for x in labels if x[1] == i]

In [4]:
# gives index to annotations per image in new column
sort_ids = df['image_name'].value_counts().sort_index()
annotation_id = [x for i in sort_ids for x in RangeIndex(i)] 
df['annotation_id'] = annotation_id

In [5]:
# building json
coco_json = {}
images_list = []
annotations_list = []
for ind, row in df.iterrows():
    bboxes = [row['bbox_x'], row['bbox_y'], row['bbox_width'], row['bbox_height']]

    images = {'id': int(row['image_name'].split('.jpg')[0]),
               'width': row['image_width'],
               'height':row['image_height'],
               'file_name': row['image_name']
             } 
             
    annotations = {
                        'id': row['annotation_id'],
                        'image_id': int(row['image_name'].split('.jpg')[0]),
                        'category_id': row['label_name'],
                        'segmentation': [],
                        'bbox': bboxes
    }               

                
    
    images_list.append(images)
    annotations_list.append(annotations)

# removes duplicates from images section of coco json
images_section = [i for n, i in enumerate(images_list) if i not in images_list[:n]]

In [6]:
coco_json['info'] = {'description': #TODO fill with function arg for video_id
    'kaDoCpLfAnA_352_354'
} 
coco_json['images'] = images_section


coco_json['annotations'] = annotations_list


coco_json['categories'] = [
{"id":0,"name":"uniformed"},
{"id":1,"name":"non_uniformed"},
{"id":2,"name":"riot_shield"},
{"id":3,"name":"gun"},
{"id":4,"name":"pepper_spray"},
{"id":5,"name":"baton"},
{"id":6,"name":"chemical_smoke"}
 ]

In [7]:
# create .json file
with open('kaDoCpLfAnA_352_354.json', 'w+') as file:
        json.dump(coco_json, file)